# Forest Fire Detection with transfer learning using ResNet50

In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras.models import load_model
from keras.applications import InceptionV3
from keras import layers
from keras import Model
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#from google.colab import drive
#drive.mount('/content/gdrive')

#train_dir = "/content/gdrive/My Drive/Escuela/Octavo Semestre/Fire_Original/Training"
#test_dir = "/content/gdrive/My Drive/Escuela/Octavo Semestre/Fire_Original/Test"

train_dir = "data/Training"
test_dir = "data/Test"

SIZE = 299

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                zoom_range=0.1,
                                horizontal_flip=True,
                                validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                subset='training',
                                                class_mode='binary')

val_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                subset='validation',
                                                class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE),
                                                color_mode='rgb',
                                                batch_size=20,
                                                class_mode='binary')

Found 39395 images belonging to 2 classes.
Found 8617 images belonging to 2 classes.


In [5]:
transfer_model = InceptionV3(include_top=True,
                        weights="imagenet",
                        classes=1000)
input = transfer_model.inputs

96116736/96112376 [==============================] - 10s 0us/step


In [9]:
transfer_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [6]:
x = transfer_model.get_layer("avg_pool").output
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(4096, activation='relu')(x)
output_layer = layers.Dense(1, activation='sigmoid')(x)

classifier = Model(input, output_layer)

for layer in classifier.layers[:-5]:
    layer.trainable = False

classifier.summary()

60[0][0]     
__________________________________________________________________________________________________
activation_63 (Activation)      (None, 17, 17, 192)  0           batch_normalization_63[0][0]     
__________________________________________________________________________________________________
activation_68 (Activation)      (None, 17, 17, 192)  0           batch_normalization_68[0][0]     
__________________________________________________________________________________________________
activation_69 (Activation)      (None, 17, 17, 192)  0           batch_normalization_69[0][0]     
__________________________________________________________________________________________________
mixed7 (Concatenate)            (None, 17, 17, 768)  0           activation_60[0][0]              
                                                                 activation_63[0][0]              
                                                                 activation_68[0][0]           

In [7]:
classifier.compile(loss="binary_crossentropy",
                optimizer=optimizers.Adam(lr=0.001),
                metrics=["accuracy"])

In [9]:
#model_path = "/content/gdrive/My Drive/Escuela/Octavo Semestre/DSDM/models/inceptionv3_model.h5"
model_path = 'models/inceptionv3_model.h5'

callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
    ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_accuracy'),
]

history = classifier.fit(train_generator, 
                        epochs=5, 
                        batch_size=40,
                        validation_data=test_generator,
                        callbacks=callbacks)

Epoch 1/5
1970/1970 [==============================] - 981s 498ms/step - loss: 0.0310 - accuracy: 0.9900 - val_loss: 0.9790 - val_accuracy: 0.7074
Epoch 2/5
1970/1970 [==============================] - 951s 482ms/step - loss: 0.0270 - accuracy: 0.9909 - val_loss: 2.7224 - val_accuracy: 0.6601
Epoch 3/5
1970/1970 [==============================] - 936s 475ms/step - loss: 0.0268 - accuracy: 0.9914 - val_loss: 1.9048 - val_accuracy: 0.6817
Epoch 4/5
1970/1970 [==============================] - 973s 494ms/step - loss: 0.0271 - accuracy: 0.9913 - val_loss: 3.5788 - val_accuracy: 0.6544


In [1]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

NameError: ignored

In [ ]:
classifier.save(model_path)

In [ ]:
result = classifier.evaluate(test_generator)
print(f"Test loss: {result[0]}, Test accuracy: {result[1]}")

431/431 [==============================] - 407s 945ms/step - loss: 0.8416 - accuracy: 0.6393
Test loss: 0.8416192531585693, Test accuracy: 0.6393176317214966
